In [1]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
   
from sklearn.metrics import auc, roc_auc_score


/kaggle/input/tabular-playground-series-oct-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-oct-2021/train.csv
/kaggle/input/tabular-playground-series-oct-2021/test.csv


In [2]:
import datatable as dt
def data_load() -> pd.DataFrame:
    direсtory_kagggle = "/kaggle/input/tabular-playground-series-oct-2021/"
    train_data = "train.csv"
    test_data = "test.csv"
    sub_data = "sample_submission.csv"
    train = pd.read_csv(direсtory_kagggle + train_data)
    test = pd.read_csv(direсtory_kagggle + test_data)
    #train = dt.fread(direсtory_kagggle + train_data).to_pandas()
    #test = dt.fread(direсtory_kagggle + test_data).to_pandas()
    sub = pd.read_csv(direсtory_kagggle + sub_data)
    
    return train, test, sub

train, test, sub = data_load()

In [3]:
train.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
0,0,0.205979,0.410993,0.176775,0.223581,0.423543,0.476140,0.413590,0.612021,0.534873,...,0,1,0,0,0,0,0,0,0,1
1,1,0.181004,0.473119,0.011734,0.213657,0.619678,0.441593,0.230407,0.686013,0.281971,...,0,1,0,0,0,0,0,0,0,1
2,2,0.182583,0.307431,0.325950,0.207116,0.605699,0.309695,0.493337,0.751107,0.536272,...,0,0,0,1,1,0,0,0,0,1
3,3,0.180240,0.494592,0.008367,0.223580,0.760618,0.439211,0.432055,0.776147,0.483958,...,0,0,0,0,1,0,0,0,0,1
4,4,0.177172,0.495513,0.014263,0.548819,0.625396,0.562493,0.117158,0.561255,0.077115,...,0,1,1,0,1,0,0,1,0,1


In [4]:
test.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284
0,1000000,0.178216,0.435617,0.010230,0.202074,0.390170,0.324221,0.221722,0.738894,0.582588,...,1,0,0,0,0,0,1,1,1,0
1,1000001,0.181250,0.476455,0.022413,0.283146,0.598020,0.349508,0.283467,0.721575,0.268990,...,0,0,0,0,0,0,0,0,0,0
2,1000002,0.159721,0.451202,0.259649,0.365274,0.594634,0.413502,0.249318,0.642339,0.411104,...,0,0,0,0,0,0,1,0,0,0
3,1000003,0.182424,0.520976,0.095344,0.327742,0.741830,0.358711,0.270077,0.601662,0.297742,...,0,0,0,0,0,1,1,0,0,0
4,1000004,0.229329,0.336513,0.023511,0.300913,0.668738,0.481586,0.545660,0.667849,0.546045,...,0,0,0,0,1,0,0,1,0,0


In [5]:
def check_na(train_df: pd.DataFrame, test_df: pd.DataFrame):
    train_na = train_df.isna().sum()
    test_na = test_df.isna().sum()
    for i in range(len(train_na)):
        if train_na[i] != 0:
            print(f"Пропущеных значений в колонке(Фича) {train_na.index[i]}" + "=== ", train_na[i])
    if train_na.sum() == 0:
        print(f"Пропущенных значений в TRAIN не обнаружено!")
    
    for i in range(len(test_na)):
        if test_na[i] != 0:
            print(f"Пропущеных значений в колонке(Фича) {test_na.index[i]}" + "=== ", test_na[i])
    if test_na.sum() == 0:
        print(f"Пропущенных значений в TEST не обнаружено!")

check_na(train, test)

Пропущенных значений в TRAIN не обнаружено!
Пропущенных значений в TEST не обнаружено!


In [6]:
from collections import Counter
def check_types_col(train_df: pd.DataFrame, test_df: pd.DataFrame):
    all_type_col_train = train_df.dtypes.unique()
    all_type_col_test = test_df.dtypes.unique()
    type_col_train = train_df.dtypes
    type_col_test = test_df.dtypes
    count_train = Counter()
    count_test = Counter()
    for type_col in all_type_col_train:
        for col_train in type_col_train:
            if col_train == type_col:
                count_train[type_col] += 1
                
    for type_col in all_type_col_test:
        for col_test in type_col_test:
            if col_test == type_col:
                count_test[type_col] += 1
                
    print("Тип и количество колонок в TRAIN", end="\n")           
    for keys, values in count_train.items():
        print(f"Тип данных: {keys} " + "--->" ,f"Количество значений: {values}")
    print("", end="\n")    
    print("Тип и количество колонок в TEST", end="\n")           
    for keys, values in count_test.items():
        print(f"Тип данных: {keys} " + "--->" ,f"Количество значений: {values}")
        
check_types_col(train, test)

Тип и количество колонок в TRAIN
Тип данных: int64 ---> Количество значений: 47
Тип данных: float64 ---> Количество значений: 240

Тип и количество колонок в TEST
Тип данных: int64 ---> Количество значений: 46
Тип данных: float64 ---> Количество значений: 240


In [7]:
memory_usage = train.memory_usage(deep=True) / 1024 ** 2
print(memory_usage.head())
print("---------------------------------------")
print("Суммарно памяти: ", memory_usage.sum())

Index    0.000122
id       7.629395
f0       7.629395
f1       7.629395
f2       7.629395
dtype: float64
---------------------------------------
Суммарно памяти:  2189.6363525390625


In [8]:
def reduce_memory_use(df: pd.DataFrame, verbose: bool=True) -> pd.DataFrame:
    num_col = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_memory_usage = df.memory_usage(deep=True).sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in num_col:
            col_min = df[col].min()
            col_max = df[col].max()
            if str(col_type)[:3] == "int":
                if col_min > np.iinfo(np.int8).min and col_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif col_min > np.iinfo(np.int16).min and col_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif col_min > np.iinfo(np.int32).min and col_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif col_min > np.iinfo(np.int64).min and col_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            elif str(col_type)[:5] == "float":
                if col_min > np.finfo(np.float16).min and col_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif col_min > np.finfo(np.float32).min and col_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                elif col_min > np.finfo(np.float64).min and col_max < np.finfo(np.float64).max:
                    df[col] = df[col].astype(np.float64)
    end_memory_usage = df.memory_usage(deep=True).sum() / 1024 ** 2
    if verbose:
        print(f"Суммарно памяти до: {start_memory_usage}", end="\n")
        print("Суммарно памяти после: ", end_memory_usage)
        
        return df
    
print("TRAIN", end="\n")
train = reduce_memory_use(train)
print("TEST", end="\n")
test = reduce_memory_use(test)
        
        
                

TRAIN
Суммарно памяти до: 2189.6363525390625
---------------------------------------------
Суммарно памяти после:  505.447509765625
TEST
Суммарно памяти до: 1091.0035400390625
---------------------------------------------
Суммарно памяти после:  252.24697875976562


In [9]:
def devide_X_y (train_df: pd.DataFrame, test_df: pd.DataFrame) -> pd.DataFrame:
    y = train_df['target']
    X = train_df.drop(['target', 'id'], axis=1)
    test = test_df.drop(['id'], axis=1)
    
    return X, y, test

X, y, test = devide_X_y(train ,test)
    

In [10]:
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
from tqdm import tqdm
def model_cat(X: pd.DataFrame, test: pd.DataFrame, y: pd.Series, n_splits=10) -> list:
    cat_prediction = np.zeros(test.shape[0])
    AUC = []
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    for trn_idx, val_idx in tqdm(kf.split(X)):
        x_train = X.iloc[trn_idx]
        y_train = y.iloc[trn_idx]
        x_valid = X.iloc[val_idx]
        y_valid = y.iloc[val_idx]
        cat_model = CatBoostClassifier()
        cat_model.fit(x_train,
                     y_train,
                     verbose=0
                     )
        cat_prediction += cat_model.predict_proba(test)[:, -1] / 5
        AUC.append(roc_auc_score(y_valid, cat_model.predict_proba(x_valid)[:, -1]))
        
    return cat_prediction, AUC



In [21]:
# score = 0.85562
#[0.8550238798256934, 0.8547225286750398, 0.855759019494479, 0.8539706371190057, 0.8551260544470253]
#Среднее значение AUC:  0.8549204239122487
# При 5 фолдах
from statistics import mean
print(AUC, end='\n')
print("Среднее значение AUC: ", mean(AUC))

[0.8550238798256934, 0.8547225286750398, 0.855759019494479, 0.8539706371190057, 0.8551260544470253]
Среднее значение AUC:  0.8549204239122487


In [11]:
# score = 0.85593
#[0.855261665751528, 0.8546279135799473, 0.852985927309796, 0.8571864863751253, 0.8552725735472613,
#0.8566601150205151, 0.8541463429627321, 0.8536059751677042, 0.8552575616606288, 0.8546144472126745]
#Среднее значение AUC:  0.8549619008587912

cat_prediction, AUC = model_cat(X, test, y, n_splits=10)

from statistics import mean
print(AUC, end='\n')
print("Среднее значение AUC: ", mean(AUC))

10it [1:49:50, 659.02s/it]

[0.855261665751528, 0.8546279135799473, 0.852985927309796, 0.8571864863751253, 0.8552725735472613, 0.8566601150205151, 0.8541463429627321, 0.8536059751677042, 0.8552575616606288, 0.8546144472126745]
Среднее значение AUC:  0.8549619008587912


In [12]:
def to_csv_file(sub: pd.DataFrame, *args: np.array) -> pd.DataFrame:
    sub['target'] = np.column_stack(args).mean(axis=1)
    sub.to_csv('Baseline_1.csv', index=False)
    
    return sub

baseline_1 = to_csv_file(sub, cat_prediction)